In [1]:
from smolagents import CodeAgent, LiteLLMModel, CodeAgent, HfApiModel,tool, ToolCallingAgent
model = LiteLLMModel(
    model_id= "ollama_chat/deepseek-r1:8b", # This model is a bit weak for agentic behaviours though
    api_base="http://localhost:11434", # replace with 127.0.0.1:11434 or remote open-ai compatible server if necessary
    api_key="", # replace with API key if necessary
    num_ctx=8192 # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [1]:
from smolagents import CodeAgent, LiteLLMModel, CodeAgent, HfApiModel,tool, ToolCallingAgent
model_1 = LiteLLMModel(
    model_id= "ollama_chat/qwen2.5-coder:latest", # This model is a bit weak for agentic behaviours though
    api_base="http://localhost:11434", # replace with 127.0.0.1:11434 or remote open-ai compatible server if necessary
    api_key="", # replace with API key if necessary
    num_ctx=80960 # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:


@tool
def mathematical_agent_tool() -> str:
    '''This is a tool to access a csv named x and give it as an input to the tool which forecasts 6 month forecasts, 
    check for words ending with .csv in the input prompt and give it as the argument x to this tool, it returns a string variable called sentence and you need to stop here
    Args: None

    returns:
    sentence: The formatted version of the 6 months forecasts 
        '''
    import pandas as pd
    import numpy as np
    from pmdarima import auto_arima

    from smolagents import ToolCallingAgent
    from smolagents.default_tools import FinalAnswerTool
    prices = pd.read_csv("iphone_price_trends_updated.csv")
    prices = prices.dropna(how='all')
    prices['Date'] = pd.to_datetime(prices['Date'], format='%Y-%m')
    df = prices[0:46]
    df.set_index('Date', inplace=True)

    y = df['New']
    X = df[['CPIAUCSL', 'Unemployment_Rate', 'Interest Rates','CPI']]

    # Fit the AutoARIMAX model
    model = auto_arima(
        y,
        exogenous=X,
        start_p=1, max_p=1,          # Constrain AR (p) to 1
        start_q=4, max_q=4,          # Constrain MA (q) to 4
        d=1,   
        seasonal=False,           # Set to True if seasonality is suspected
        trace=True,               # Display progress of parameter search
        suppress_warnings=True,
        stepwise=True             # Perform stepwise parameter selection
    )
    model.fit_with_exog_
    # Print the selected ARIMAX order
    print(f"Best ARIMAX order: {model.order}")
    print(f"Best Seasonal Order: {model.seasonal_order}")

    # Forecast next 6 months (provide future values for exogenous variables)
    future_dates = pd.date_range(start='2024-07-01', periods=6, freq='M')
    future_exog = pd.DataFrame({
        'CPIAUCSL': prices['CPIAUCSL'][46:].values,
        'Unemployment_Rate': prices['Unemployment_Rate'][46:].values,
        'Interest Rates': prices['Interest Rates'][46:].values,
        'CPI': prices['CPI'][46:].values,
    }, index=future_dates)
    print(future_exog)

    # Make predictions
    forecast = model.predict(n_periods=6, exogenous=future_exog)

    # Combine forecast with future dates
    forecast_df = pd.DataFrame({'Date': future_dates, 'Forecasted New Price': forecast})
    forecast_df.set_index('Date', inplace=True)

    # Display forecasted prices
    future_exog = pd.DataFrame({
        'CPIAUCSL': prices['CPIAUCSL'][46:],
        'Unemployment_Rate': prices['Unemployment_Rate'][46:],
        'Interest Rates': prices['Interest Rates'][46:],
        'CPI': prices['CPI'][46:],
    }, index=future_dates)
    forecast = model.predict(n_periods=6, exogenous=future_exog)
    sentence = ', '.join([f"{index.strftime('%Y-%m-%d')} the price is {value}" for index, value in forecast.items()])
    return sentence


In [3]:
mathematical_agent = CodeAgent(tools=[mathematical_agent_tool], model=model_1,  add_base_tools=False, max_steps = 5, verbosity_level = 5,additional_authorized_imports = ["*"], name = 'mathematical_agent', description = "access ###'iphone_price_trends_updated.csv' in the current working directory and give the name of the csv as input to mathematical_agent_tool, dont modify the final sentence variable the out should be the sentence as it is" )

Caution: you set an authorization for all imports, meaning your agent can decide to import any package it deems 
necessary. This might raise issues if the package is not installed in your environment. 0

In [4]:
sentence = mathematical_agent.run(f"Call the tool, return the execution logs as the sentence variable")

╭───────────────────────────────────────── New run - mathematical_agent ──────────────────────────────────────────╮
│                                                                                                                 │
│ Call the tool, return the execution logs as the sentence variable                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:latest ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to call the `mathematical_agent_tool` and return its output as the 'sentence' variable. This tool  
requires a CSV file as input, but since there is no specific file mentioned in this query, I will assume that the  
function will work without any arguments.                                                                          
Code:                                                                                                              
```py                                                                                                              
sentence = mathematical_agent_tool()                                                                               
final_answer({'answer': sentence})                                                                                 
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  sentence = mathematical_agent_tool()                                                                             
  final_answer({'answer': sentence})                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Performing stepwise search to minimize aic
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=503.368, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=505.209, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=504.982, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=506.087, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=506.464, Time=0.03 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.164 seconds
Best ARIMAX order: (0, 1, 0)
Best Seasonal Order: (0, 0, 0, 0)
            CPIAUCSL  Unemployment_Rate  Interest Rates       CPI
2024-07-31   314.131                4.2        1.664190  3.701742
2024-08-31   314.851                4.1        1.581581  4.146461
2024-09-30   315.564                4.1        1.480148  3.698695
2024-10-31   316.449                4.2        1.958417  3.226070
2024-11-30   317.603                4.1        1.824369  3.234511
2024-12-31   319.086               

/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:1

Out - Final answer: {'answer': '2024-08-01 the price is 299.3333333333333, 2024-09-01 the price is 
278.66666666666663, 2024-10-01 the price is 257.99999999999994, 2024-11-01 the price is 237.33333333333326, 
2024-12-01 the price is 216.66666666666657, 2025-01-01 the price is 195.9999999999999'}

[Step 0: Duration 8.79 seconds| Input tokens: 2,076 | Output tokens: 81]

In [4]:
# After executing a tool and getting its result
tool_result = mathematical_agent.execute_tool_call('mathematical_agent_tool',{})




Performing stepwise search to minimize aic
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=503.368, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=505.209, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=504.982, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=506.087, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=506.464, Time=0.03 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.150 seconds
Best ARIMAX order: (0, 1, 0)
Best Seasonal Order: (0, 0, 0, 0)
            CPIAUCSL  Unemployment_Rate  Interest Rates       CPI
2024-07-31   314.131                4.2        1.664190  3.701742
2024-08-31   314.851                4.1        1.581581  4.146461
2024-09-30   315.564                4.1        1.480148  3.698695
2024-10-31   316.449                4.2        1.958417  3.226070
2024-11-30   317.603                4.1        1.824369  3.234511
2024-12-31   319.086               

/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/kronos/anaconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:1

In [5]:
mathematical_agent.state = tool_result

In [6]:
mathematical_agent.state

'2024-08-01 the price is 299.3333333333333, 2024-09-01 the price is 278.66666666666663, 2024-10-01 the price is 257.99999999999994, 2024-11-01 the price is 237.33333333333326, 2024-12-01 the price is 216.66666666666657, 2025-01-01 the price is 195.9999999999999'

In [10]:
print(mathematical_agent.memory.replay)

<bound method AgentMemory.replay of <smolagents.memory.AgentMemory object at 0x7a314b38dd00>>


In [ ]:


# Create the model

@tool
def market_preds(sentence: str) -> str:
    """
    Returns the interpretation of the forecasts from the purview of the real word events

    Args:
        sentence: the forecasts for 6 months
    """
    description = """
    This is about a product called Apple Iphone 12 128 GB, you will take the the forecast of the price in dollars and give the macroeconomic or product related or Geopolitical or other events that influenced the increase or decrease, refer internet for the events"""
    import os
    import google.generativeai as genai

    genai.configure(api_key="*")
    generation_config = {
        "temperature": 0.7,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 4096,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name="gemini-2.0-pro-exp-02-05",
        generation_config=generation_config,
    )

    chat_session = model.start_chat(
        history=[]
    )
    events = chat_session.send_message(f"Give me all events from February 2020 to February 2025 in month and year format that affect apple's supply chain or events that can affect the price of a iphone, including supply chain , quality issues, competitor launch, or new product launch, geopolitics, tariffs, China, taiwan, antitrust and quality issues")
    response = chat_session.send_message(f"{sentence} is the forecast of iphone 12 128GB for 6 months and {events.text} is the events that can affect the price of an iphone, evaluate the forecast and events and associate each forecast with the corresponding event and give an elaborate association of events and price forecast ")
    return response.text



In [ ]:
market_agent = CodeAgent(tools=[market_preds], model=model_1,  add_base_tools=False, max_steps = 5, verbosity_level = 5, name = "market_agent", additional_authorized_imports=["*"],description = 'This agent helps in doing market research for the product described and returns the interpreted message')


In [ ]:
market = market_agent.run(f"{sentence} is the input to the market_preds and the forecast of iphone 12 128GB for 6 months and the events that can affect the price of an iphone, evaluate the forecast and events and associate each forecast with the corresponding event and give an elaborate association of events and price forecast , t, return the ###'execution logs' as Final Answer")

In [ ]:
from smolagents import ToolCallingAgent,CodeAgent
from smolagents.default_tools import FinalAnswerTool,DuckDuckGoSearchTool, VisitWebpageTool

web_agent = CodeAgent(tools=[DuckDuckGoSearchTool(),VisitWebpageTool()], model=model_1,  add_base_tools=True, max_steps = 5, verbosity_level = 5 , additional_authorized_imports=["*"], name = "web_agent", description = "Write code to go to duckduckgo and for iphone shipments and redo the same iphone revenue share, sumamrize the search results be specific with numbers and years , try to get years 2020 to 2025 and print the summary")

In [ ]:
web_search = web_agent.run(f"""Write code to go to duckduckgo and search for quanitty of iphone shipments and redo the same iphone revenue share, sumamrize the search results be specific with numbers and years , try to get years 2020 to 2025 and print as a complete sentence by sumamrizing the search results of each year with any additional information with the quantities and revenue for each year""")

In [ ]:
@tool
def recommend_tool(sentence: str, web_search: str, market:list) -> str:
    """
    Returns the pricing optimization advice

    Args:
        sentence: the forecasts for 6 months
        web_search: The quantities and revenue associated with the iphone sales
        market: The events that affect the iphone price and their mapping with the forecast
    """
    description = """
    This is about a product called Apple Iphone 12 128 GB, you will take the the forecast of the price in dollars and give the macroeconomic or product related or Geopolitical or other events that influenced the increase or decrease, refer internet for the events"""
    import os
    import google.generativeai as genai

    genai.configure(api_key="*")
    generation_config = {
        "temperature": 0.1,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 4096,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name="gemini-2.0-pro-exp-02-05",
        generation_config=generation_config,
    )

    chat_session = model.start_chat(
        history=[]
    )
    total = sentence + web_search +' '.join(market)
    response = chat_session.send_message(f"{total} is the output of all the three agents, the forecasts for the next 6 months , the events that might have impacted the iphone price and historical revenue and quantity numbers, now based on all these try price elasticity and other price optimisation and dynamic pricing theories and give a condensed recommendation of the pricing strategy along with the future expectations of the price ")
    return response.text

In [ ]:
recommend_agent = CodeAgent(tools=[recommend_tool], model=model_1,  add_base_tools=False, max_steps = 5, verbosity_level = 5, name = "recommend_agent", additional_authorized_imports=["*"],description = 'This agent helps in consolidating the outputs of the other three and gives the pricing strategy recommendations')


In [ ]:
recommend_agent.run("{Given are the output of all the three agents, the forecasts for the next 6 months , the events that might have impacted the iphone price and historical revenue and quantity numbers, now based on all these try price elasticity and other price optimisation and dynamic pricing theories and give a condensed recommendation of the pricing strategy along with the future expectations of the average price for  all iphone models in common and also the risks associated, geopolitical, social, economic, macro economic, monetary summarize in one paragraph")

In [ ]:

import gradio as gr

# Define the function that runs the agents and generates the recommendation
def process_prompt(prompt):
    # Run the mathematical agent
    sentence = mathematical_agent.run(prompt)
    
    # Run the market agent
    market = market_agent.run(f"{sentence} is the input to the market_preds...")
    
    # Run the web search agent
    web_search = web_agent.run(market)
    
    # Run the recommendation agent
    recommendation = recommend_agent.run("Given are the output of all the three agents, the forecasts for the next 6 months , the events that might have impacted the iphone price and historical revenue and quantity numbers, now based on all these try price elasticity and other price optimisation and dynamic pricing theories and give a condensed recommendation of the pricing strategy along with the future expectations of the average price for  all iphone models in common and also the risks associated, geopolitical, social, economic, macro economic, monetary summarize in one paragraph")
    
    # Return the outputs from the agents
    return sentence, market, web_search, recommendation

# Create the Gradio interface




In [ ]:
with gr.Blocks() as demo:
    # Define the input component
    prompt = gr.Textbox(label="Prompt", lines=5)  # Allow for larger text input
    
    # Define the output components
    sentence_output = gr.Textbox(label="Mathematical Agent Output", lines=5)  # Allow for larger text output
    market_output = gr.Textbox(label="Market Agent Output", lines=5)  # Allow for larger text output
    web_search_output = gr.Textbox(label="Web Search Agent Output", lines=5)  # Allow for larger text output
    recommendation_output = gr.Textbox(label="Recommendation", lines=5)  # Allow for larger text output
    
    # Define the button to trigger the function
    submit_btn = gr.Button("Submit")
    
    # Define the event listener for the button click
    submit_btn.click(
        fn=process_prompt,
        inputs=prompt,
        outputs=[sentence_output, market_output, web_search_output, recommendation_output]
    )

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(show_error=True)